# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
 
from doctest import testmod    
def priest(sex, age, breathe, sat, heart_rate, sys, tem, aler, ins, perf):
        
    """
    The priest function takes 10 different parameters  provided on the MDCalc website to compute a risk percentage
    that is returned as a numeric value between 0 and 1.
    
    >>> print(priest('Male', 45, 48.0, 0.86, 180.0, 115.0, 21.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
    0.49
    >>> print(priest('Male', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
    0.46
    >>> print(priest('Female', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
    0.38
    
    """    
    
    score = 0
    percentage = {0:1,1:1,2:2,3:2,4:3,5:9,6:15,7:18,8:22,9:26,10:29,11:34,12:38,13:46,14:47,15:49,16:55}
    result = 0.0
    perf_st = {"unrestricted normal activity":0, "limited strenuous activity, can do light activity":1,"limited activity, can self-care":2, "limited self-care":3, "bed/chair bound, no self-care":4}
    sex = sex.lower()
    if(sex != "male" and sex != "female"):
        raise ValueError("Invalid Input. The sex can be either male or female")
    else:
        if sex == "male":
            score += 1
            
            
    if(age >= 16 and age <= 49):
        score += 0
    elif(age >= 50 and age <= 65):
        score += 2
    elif(age >= 66 and age <= 80):
        score += 3
    elif(age > 80):
        score += 4
    else:
        
        raise ValueError("Invalid Input. age must be greater than 16")
        
    if(breathe >= 9 and breathe <= 11):
        score += 1
    elif(breathe < 9):
        score += 3
    elif(breathe >= 12 and breathe <= 20):
        score += 0
    elif(breathe >=21 and breathe <= 24):
        score += 2
    elif(breathe > 24):
        score += 3
    else:
        raise ValueError("Invalid Input. please enter valid respiratory rate")
        
    if(sat > 95):
        score+= 0
    elif(sat >= 94 and sat <= 95):
        score += 1
    elif(sat >= 92 and sat <= 93):
        score += 2
    elif(sat <= 92):
        score += 3
        
    if(heart_rate >= 41 and heart_rate <= 50):
        score += 1
    elif(heart_rate >= 51 and heart_rate <= 90):
        score += 0
    elif(heart_rate >= 91 and heart_rate <= 110):
        score += 1
    elif(heart_rate >= 111 and heart_rate <= 130):
        score += 2
    elif(heart_rate > 130):
        score += 3
    else:
        pass#raise ValueError("Invalid Input. please enter valid heart rate")
        
    if(sys < 91):
        score += 3
    elif(sys >= 91 and sys <= 100):
        score += 2
    elif(sys >= 101 and sys <= 110):
        score += 1 
    elif(sys >= 111 and sys <= 219):
        score += 0
    elif(sys >= 219):
        score += 3
    else:
        raise ValueError("Invalid Input. please enter valid systolic bp")
        
    if(tem < 35.1):
        score += 3
    elif(tem >= 35.1 and tem <= 36.0):
        score += 1
    elif(tem >= 36.1 and tem <= 38.0):
        score += 0
    elif(tem >= 38.1 and tem <= 39.0):
        score += 1
    elif(tem > 39.0):
        score += 2
    else:
        raise ValueError("Invalid Input. please enter valid temperature")
        
    aler = aler.lower()
    if(aler != "alert" and aler != "confused or not alert"):
        raise ValueError("Invalid Input. The alertness value can be either elert or confused or not alert")
    else:
        if aler == "confused or not alert":
            score += 3
            
    ins = ins.lower()
    if(ins != "air" and ins != "supplemental oxygen"):
        raise ValueError("Invalid Input. The inspired oxygen should be either Air or supplemental oxygen")
    else:
        if ins == "supplemental oxygen":
            score += 2
            
    perf = perf.lower()
    if perf not in perf_st:
        raise ValueError("Invalid Input. Please give valid performance status")
    else:
        score += perf_st[perf]
        
    if(score >= 17 and score <= 25):
        result = 0.59
    elif(score >= 26):
        result = 0.99
    else:
        result = percentage[score] / 100
        
    return(result)
ct = 19
if __name__ == '__main__':
    testmod(name ='print', verbose = True)

Trying:
    print(priest('Male', 45, 48.0, 0.86, 180.0, 115.0, 21.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
Expecting:
    0.49
ok
Trying:
    print(priest('Male', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
Expecting:
    0.46
ok
Trying:
    print(priest('Female', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
Expecting:
    0.38
ok
1 items had no tests:
    print
1 items passed all tests:
   3 tests in print.priest
3 tests in 2 items.
3 passed and 0 failed.
Test passed.


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [2]:
import requests
def find_hospital(age, sex, risk):
    
    """
    The find_hospital takes age sex and risk as parameters and outputs the name of the hospital using REST API.
    >>> print(find_hospital(45,"male", 0.45))
    Emory Dunwoody Medical Center

    >>> print(find_hospital(16,"female",0.38)) 
    Hughes Spalding Childrens Hospital

    >>> print(find_hospital(55,"male", 0.12))
    Southwest Hospital and Medical Center

    >>> print(find_hospital(28,"female", 0.87))
    Emory Dunwoody Medical Center

    >>> print(find_hospital(10,"female", 0.22))
    Childrens Healthcare of Atlanta at Scottish Rite
    
    
    
    """
    try:
        api_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age="+str(age)+"&sex="+sex+"&risk_pct="+str(risk)
        response = requests.get(api_url)
        dict_ = response.json()
        result = dict_['hospital']
        return(result)
    except:
        return(None)


if __name__ == '__main__':
    testmod(name ='print', verbose = True)

Trying:
    print(find_hospital(45,"male", 0.45))
Expecting:
    Emory Dunwoody Medical Center
ok
Trying:
    print(find_hospital(16,"female",0.38)) 
Expecting:
    Hughes Spalding Childrens Hospital
ok
Trying:
    print(find_hospital(55,"male", 0.12))
Expecting:
    Southwest Hospital and Medical Center
ok
Trying:
    print(find_hospital(28,"female", 0.87))
Expecting:
    Emory Dunwoody Medical Center
ok
Trying:
    print(find_hospital(10,"female", 0.22))
Expecting:
    Childrens Healthcare of Atlanta at Scottish Rite
ok
Trying:
    print(priest('Male', 45, 48.0, 0.86, 180.0, 115.0, 21.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
Expecting:
    0.49
ok
Trying:
    print(priest('Male', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
Expecting:
    0.46
ok
Trying:
    print(priest('Female', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity'))
Expecting:
    0.38
ok

## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [3]:
import requests
from bs4 import BeautifulSoup

def get_address(hospital):
    
    """
    The get_address function takes hospital name as parameter and returns its location using beautiful.
    
    >>> print(get_address("Emory Dunwoody Medical Center"))
    4500 north shallowford road

    >>> print(get_address("Hughes Spalding Childrens Hospital"))
    1711 tullie circle ne

    >>> print(get_address("Childrens Healthcare of Atlanta at Scottish Rite"))
    1001 johnson ferry rd
    
    
    """
    
    url = "https://www.officialusa.com/stateguides/health/hospitals/georgia.html"
    html = requests.get(url).content
    data = BeautifulSoup(html, 'html.parser')
    table = data.find(id='myTable')
    #print(type(table))
    #table_body = table.find('tbody')
    #print(table)
    rows = table.find_all('tr')
    #print(rows)
    
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        #print(cols)
        data.append([ele for ele in cols if ele])
    data.pop(0)
    struc = {}
    for info in data:
        if(info[0].lower() == "atlanta"):
            if info[1].lower() in struc:
                struc[info[1].lower()].append(info[3].lower())
            else:
                struc[info[1].lower()] = [info[3].lower()]
    
    if(hospital.lower() in struc):
        if(len(struc[hospital.lower()]) > 1):
            raise KeyError("More than one matching hospital in Atlanta with the same name")
        else:
            return(struc[hospital.lower()][0])
    else:
        raise KeyError("Hospital is not found!!!")



if __name__ == '__main__':
    testmod(name ='print', verbose = True)

Trying:
    print(find_hospital(45,"male", 0.45))
Expecting:
    Emory Dunwoody Medical Center
ok
Trying:
    print(find_hospital(16,"female",0.38)) 
Expecting:
    Hughes Spalding Childrens Hospital
ok
Trying:
    print(find_hospital(55,"male", 0.12))
Expecting:
    Southwest Hospital and Medical Center
ok
Trying:
    print(find_hospital(28,"female", 0.87))
Expecting:
    Emory Dunwoody Medical Center
ok
Trying:
    print(find_hospital(10,"female", 0.22))
Expecting:
    Childrens Healthcare of Atlanta at Scottish Rite
ok
Trying:
    print(get_address("Emory Dunwoody Medical Center"))
Expecting:
    4500 north shallowford road
ok
Trying:
    print(get_address("Hughes Spalding Childrens Hospital"))
Expecting:
    1711 tullie circle ne
ok
Trying:
    print(get_address("Childrens Healthcare of Atlanta at Scottish Rite"))
Expecting:
    1001 johnson ferry rd
ok
Trying:
    print(priest('Male', 45, 48.0, 0.86, 180.0, 115.0, 21.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity

## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [4]:
import csv

def process_people(file_name):
    
    """
    process_people function takes filename as input and evaluate 100 examples and creates a struction with lists 
     like { patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}
    
    
    """
    
    with open('/data/'+file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        next(csv_reader, None)
        dict_ = {}
        for row in csv_reader:
            risk = priest(row[1], int(row[2]), int(row[3]), float(row[4]), int(row[5]), int(row[6]), float(row[7]), row[8], row[9], row[10])
            hospital = find_hospital(row[2], row[1], risk)
            address = get_address(hospital)
            dict_[row[0]] = [row[1],int(row[2]),float(row[3]),float(row[4]),float(row[5]),float(row[6]),float(row[7]),row[8],row[9],row[10],hospital,address.upper()]
            #print(row)
    return(dict_)




## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [5]:
import json
f = open('/data/people_results.json')
data = json.load(f)
for i in data.values():
    del i[10]

result = process_people("people.psv")




In [6]:
for li in data.keys():

    if(data[li] == result[li]):
        
        ct += 1
print(ct,"are correct out of 100!")

100 are correct out of 100!


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [7]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main ddf2c59] Submitting the midterm
 1 file changed, 4 insertions(+), 4 deletions(-)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (4/4), 418 bytes | 418.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:vikas060/hds5210-2022.git
   d05ee11..ddf2c59  main -> main
